In [17]:
import numpy as np
import pickle as pkl

def load_data(path):
    with open(path, 'rb') as f:
        data = pkl.load(f)
    return data

# Load the data
Interaction_matrices = load_data('interaction_matrices.pkl')
spectral_data = load_data('spectra_dataset.pkl')

print(Interaction_matrices.shape)
print(spectral_data.shape)

torch.Size([10000, 6, 6])
torch.Size([10000, 20000])


In [38]:
import torch

# reshaping for LSTM - added one dimension at the end
spectral_data_tensor = torch.tensor(spectral_data.unsqueeze(-1), dtype=torch.float32)
matrices_tensor = torch.tensor(Interaction_matrices.view(-1, 36), dtype=torch.float32)

new_tensor = spectral_data_tensor[:4000]
new_matrices = matrices_tensor[:4000]
print(new_tensor.shape)
print(new_matrices.shape)

print(matrices_tensor.shape)
print(spectral_data_tensor.shape)

print(spectral_data.dtype)
print(Interaction_matrices.dtype)

# already float32 

/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_18909/1288698792.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spectral_data_tensor = torch.tensor(spectral_data.unsqueeze(-1), dtype=torch.float32)


torch.Size([4000, 20000, 1])
torch.Size([4000, 36])
torch.Size([10000, 36])
torch.Size([10000, 20000, 1])
torch.float32
torch.float32


/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_18909/1288698792.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrices_tensor = torch.tensor(Interaction_matrices.view(-1, 36), dtype=torch.float32)


In [39]:
from sklearn.model_selection import train_test_split

spec_train, spec_test, matrix_train, matrix_test = train_test_split(new_tensor, new_matrices, test_size=0.2, random_state=42)
spec_test, spec_val, matrix_test, matrix_val = train_test_split(spec_test, matrix_test, test_size=0.5, random_state=42)


# reshaped for LSTM
print(spec_train.shape)
print(spec_test.shape)
print(spec_val.shape)

print(matrix_train.shape)
print(matrix_test.shape)
print(matrix_val.shape)

torch.Size([3200, 20000, 1])
torch.Size([400, 20000, 1])
torch.Size([400, 20000, 1])
torch.Size([3200, 36])
torch.Size([400, 36])
torch.Size([400, 36])


In [40]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64
# pytorch dataset and loaders
train_dataset = TensorDataset(torch.Tensor(spec_train), torch.Tensor(matrix_train))
val_dataset = TensorDataset(torch.Tensor(spec_val), torch.Tensor(matrix_val))
test_dataset = TensorDataset(torch.Tensor(spec_test), torch.Tensor(matrix_test))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



In [41]:
for data in train_loader:
    print(data[0].shape)
    print(data[1].shape)
    print(batch_size)
    break

torch.Size([64, 20000, 1])
torch.Size([64, 36])
64


In [42]:
import torch
import torch.nn as nn   

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.regressor = nn.Linear(64, 36)

    def forward(self, x):
        # forward passing
        lstm_out, _ = self.lstm(x)

        # only last time step
        lstm_out = lstm_out[:, -1, :]

        # pass output to lstm to fully connected later to predict all 36 values
        output = self.regressor(lstm_out)
        return output


In [43]:
import torch
import torch.nn as nn
import torch.optim as optim

model = LSTM()

criteria = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [45]:
import time
from torch.profiler import profile, record_function, ProfilerActivity
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.0001,
    "architecture": "LSTM",
    "dataset": "4000-part-spectra",
    "epochs": 10,
    }
)

def train_model(model, data_loader, criterion, optimizer, epochs):
    model.train()
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        for epoch in range(epochs):
            print("inside the loop")
            print(f"started at {time.ctime()}")
            total_loss = 0
            for spectra, labels in data_loader:
                optimizer.zero_grad()
                with record_function("model_forward"):
                    output = model(spectra)
                with record_function("calc_loss"):
                    loss = criterion(output, labels)
                with record_function("backprop"):
                    loss.backward()
                optimizer.step()
                total_loss += loss.item() * spectra.size(0)
            
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(data_loader.dataset)}')
            print(f"ended at {time.ctime()}")
    print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

train_model(model, train_loader, criteria, optimizer, epochs=10)

# 2m22seconds for 800 dataset epoch 1 batch size 32 total loss ~6.5 lr = 0.001
# 2m22seconds for 800 dataset epoch 1 batch size 64 loss ~7.196 lr = 0.001



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sanazkazeminia97 (sanaz_team). Use `wandb login --relogin` to force relogin


inside the loop
started at Thu Apr 25 16:00:47 2024


STAGE:2024-04-25 16:00:47 18909:7383288 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
